In [2]:
from selenium import webdriver
from time import sleep
import pandas as pd
from io import StringIO
from functools import partial
import matplotlib.pyplot as plt
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common import NoSuchElementException,StaleElementReferenceException
from matplotlib import style

print(plt.style.available)
%matplotlib inline

driver=webdriver.Edge() #打开浏览器
url='https://tj.esf.fang.com/house-a037-b0154/' #房源链接
driver.get(url)

['Solarize_Light2', '_classic_test_patch', '_mpl-gallery', '_mpl-gallery-nogrid', 'bmh', 'classic', 'dark_background', 'fast', 'fivethirtyeight', 'ggplot', 'grayscale', 'petroff10', 'seaborn-v0_8', 'seaborn-v0_8-bright', 'seaborn-v0_8-colorblind', 'seaborn-v0_8-dark', 'seaborn-v0_8-dark-palette', 'seaborn-v0_8-darkgrid', 'seaborn-v0_8-deep', 'seaborn-v0_8-muted', 'seaborn-v0_8-notebook', 'seaborn-v0_8-paper', 'seaborn-v0_8-pastel', 'seaborn-v0_8-poster', 'seaborn-v0_8-talk', 'seaborn-v0_8-ticks', 'seaborn-v0_8-white', 'seaborn-v0_8-whitegrid', 'tableau-colorblind10']


In [3]:
string_list=[]
i=0
#数据获取
while i<20:
    table = driver.find_element(By.CLASS_NAME,'shop_list.shop_list_4')
    # Extract table rows
    rows = table.find_elements(By.TAG_NAME, 'dl')
    # Initialize lists to store data
    data = []
    # Loop through rows and extract data
    for row in rows:
        cells = row.find_elements(By.TAG_NAME, 'dd')
        if cells:  # Skip rows without data (e.g., headers)
            row_data = [cell.text for cell in cells]
            data.append(row_data)
    df = pd.DataFrame(data)
    string_list.append(df)
    #driver.refresh()
    try:
        next_page=driver.find_element(By.LINK_TEXT,'下一页')
        next_page.click()
        i+=1
    except NoSuchElementException:
        break


In [4]:
def split_info(string_list,index_num):
    info = string_list[0][0][index_num].split("\n")
    price_info = string_list[0][1][index_num].split("\n")
    #数据清洗
    if info[1]=="双重验真":#异常值处理
        title = info[0]
        address = info[3]
        price = price_info[0]
        unit_price = price_info[1]
        type =  info[2].split("|")[0]
        square =  info[2].split("|")[1]
        floor =  info[2].split("|")[2]
        direction =  info[2].split("|")[3]
        district =  info[4]
        traffic =  info[-1]
        if len(info[2].split("|")) ==6:#异常值处理
            date = info[1].split("|")[4]
        else:
            date = None
    else:
        title = info[0]
        address = info[2]
        price = price_info[0]
        unit_price = price_info[1]
        type =  info[1].split("|")[0]
        square =  info[1].split("|")[1]
        floor =  info[1].split("|")[2]
        direction =  info[1].split("|")[3]
        district =  info[3]
        traffic =  info[-1]
        if len(info[1].split("|")) ==6:#异常值处理
            date = info[1].split("|")[4]
        else:
            date = None
    return [title,address,price,unit_price,type,square,floor,direction,date,district,traffic]


In [5]:
df = pd.DataFrame(columns=["标题","小区","总价","单价","户型","面积","层型","朝向","建成时间","区域","交通信息"])
for i in range(len(string_list)):#遍历子列表 写入数据
    try:
        for j in range(len(string_list[i])):
            if j>59:
                i+=1
            infomation = split_info(string_list,j)
            new_row = pd.DataFrame([infomation], columns=df.columns)
            df = pd.concat([df, new_row], ignore_index=True)
    except Exception as e:
        print(f"处理第 {i+1} 个子列表的第{j+1}条数据时出错: {e}")

In [8]:
# 删除“建成时间”列为 None 的数据
df = df.dropna(subset=['建成时间'])

In [9]:
df

,标题,小区,总价,单价,户型,面积,层型,朝向,建成时间,区域,交通信息
2,和平1片 鞍山道 公产 可以贷款 无无过户费 3天下本,新疆路二号公寓,138万,55200元/㎡,1室0厅,25㎡,顶层 （共3层）,西北向,1901年建,劝业场街-和平区万全道与新疆路交口,距1号线鞍山道站约313米
4,急和平门槛房 能贷款 集体户能用 7天下本,沈阳道小区,105万,82095元/㎡,1室0厅,12.79㎡,底层 （共2层）,西南向,1940年建,劝业场街-沈阳道与辽宁路交口,距1号线营口道站约605米
5,拥金1%鞍山道南北通透精装大两室 采光视野无遮挡 私产满五年,信德楼,260万,46387元/㎡,2室1厅,56.05㎡,中层 （共5层）,南北向,2018年建,劝业场街-陕西路108号,距1号线鞍山道站约572米
6,7天下本 和平一 能贷款 集体户能用 0中间费0过户非,赤峰道小区,110万,55000元/㎡,1室0厅,20㎡,低层 （共6层）,东向,1940年建,劝业场街-和平区赤峰道,距1号线营口道站约419米
7,"鞍山道有座位,随时上學)和平一片(中介費税費全免)三天拿房本",新疆路二号公寓,138万,55200元/㎡,1室0厅,25㎡,顶层 （共3层）,西北向,1942年建,劝业场街-和平区万全道与新疆路交口,距1号线鞍山道站约313米
...,...,...,...,...,...,...,...,...,...,...,...
1195,和平3片 四平东道学片房(中介費税費全免)可公易贷/款,哈尔滨道,99万,53225元/㎡,1室0厅,18.6㎡,顶层 （共2层）,西北向,1963年建,劝业场街-(和平) 哈尔滨道,距1号线营口道站约432米
1196,和平一片 哈密道小學 可转學 落沪 贷款 门槛房 太和里,万全道,85万,39534元/㎡,1室0厅,21.5㎡,底层 （共2层）,西南向,2006年建,劝业场街-(和平) 万全道,距1号线鞍山道站约579米
1197,和平二片 忠芯小學 有學籍 可贷款 能落户 丰收里 兆丰路,松月西里,158万,73080元/㎡,1室1厅,21.62㎡,底层 （共2层）,南北向,2006年建,劝业场街-河南路92号,距1号线营口道站约527米
1198,万全小學有學籍 一室价买两室 私产可贷款 庆有西里 福方里,蒙古路,230万,49313元/㎡,2室1厅,46.64㎡,高层 （共6层）,北向,2006年建,劝业场街-(和平) 和平区蒙古路,距1号线营口道站约678米


In [12]:
df.to_csv("queyechang_esf_data.csv")